In [ ]:
# @title Libraries
import os
import tensorflow as tf
from google.colab import drive
import pandas as pd
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import math
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor,ExtraTreesRegressor
import numpy as np
import statistics
from sklearn.feature_selection import VarianceThreshold
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from hyperopt import fmin, tpe, hp, Trials
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler , MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import r_regression, f_regression, mutual_info_regression
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import PredictionErrorDisplay
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
import time
import statistics
from itertools import product
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2
# from mlxtend.feature_selection import FSSearch
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from itertools import product

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import KFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title Variable
# @title
variable_name = "" # @param {type:"string"}
################ PARAMETERS ####################




numberOfFold = 5
DATASET = ['SNMMI_CHALLENGE_TRAINING_V22OCT2023_Update']
################################################
import warnings
result_path = '/content/drive/MyDrive/projects/reserach/Challenge_SNMMI_2023/Results/Amirali & Nima/22 12 2023/'
def append_row(df, row):
    return pd.concat([
                df,
                pd.DataFrame([row], columns=row.index)]
          ).reset_index(drop=True)

In [ ]:
# @title Load data
df = pd.read_excel("/content/drive/MyDrive/projects/reserach/Challenge_SNMMI_2023/Data/SNMMI_CHALLENGE_TRAINING_V22OCT2023_Update Oct 26.xlsx", index_col=0)
dftest = pd.read_excel("/content/drive/MyDrive/projects/reserach/Challenge_SNMMI_2023/Data/SNMMI_CHALLENGE_TESTING_V01112023.xlsx", index_col = 0)
X_mainL1_test = dftest

# df.drop(["Event"], axis=1, inplace=True)

filtered_df_s = df[df['Event'] != 0]
target_column1 = filtered_df_s["Outcome_PFS"]

Y_mainL1 = target_column1
sup_indxs = Y_mainL1.index

X_mainL1 = df.loc[sup_indxs].drop(columns=['Event', 'Outcome_PFS'])


In [ ]:
# @title Random Forest Regressor


# Constants and initialization
reg_name = 'Random Forest Regressor'
lowest_test_mae_RandomForestRegressor = float('inf')


iteration_count = 0
results7 = {'Approach':[],
            "Regressor":[],
    "Feature_Selection":[],
    'Kfold':[],
    'Scaler' :[],
    'TEST_MSE':[],
    'TRAIN_MSE': []
}


X_train = X_mainL1
y_train = Y_mainL1
X_test = X_mainL1_test

#Define the parameter grid for the grid search
param_grid = {
'model__n_estimators': [50, 100, 200],
'model__max_depth': [2, 5, 10],
'model__min_samples_split': [2, 5, 10],
'model__min_samples_leaf': [1, 2, 4],
'model__max_features': ['sqrt', 'log2', None]
}

# Create an RandomForest regressor
reg = RandomForestRegressor()

# Fit the imputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')


# Scaler
scaler = MinMaxScaler()

#pca
pca = PCA(n_components=2)


#Feature Selection
feat_selec = SelectKBest(r_regression, k=50)

# Pipeline
pipe = Pipeline(steps=[("imputer", imputer), ("Scaler", scaler),("feat_select",feat_selec),("model", reg) ])

#Define k_fold for Inner Loop
cv1 = KFold(n_splits=5, shuffle=True, random_state=0)


# Create a grid search object with 5-fold cross-validation
grid_search = GridSearchCV(pipe, param_grid, cv=cv1, scoring='neg_mean_squared_error')

# # Fit the grid search object to the training data
grid_search.fit(X_train, y_train)


#Define k_fold for Outer Loop
cv2 = KFold(n_splits=5, shuffle=True, random_state=0)

cv_results = cross_validate(grid_search, X_train, y_train, cv=cv2, scoring='neg_mean_squared_error', return_train_score=True)


#MSE
testSC_mse = np.absolute(cv_results['test_score'])
trainSC_mse = np.absolute(cv_results['train_score'])

testM_mse = np.mean(testSC_mse)
trainM_mse = np.mean(trainSC_mse)

testD_mse = np.std(testSC_mse)
trainD_mse = np.std(trainSC_mse)







print('Feature_selction:', 'rr')
print("TEST MSE : " + str(testM_mse)+ " ± " + str(testD_mse))
print("TRAIN MSE : " + str(trainM_mse)+ " ± " + str(trainD_mse))
iteration_count += 1
print(iteration_count)

#Make list for the final datasheet
#Make list for the final datasheet
results7['Approach']. append(False)
results7["Regressor"].append(reg_name)
results7["Feature_Selection"].append('rr')
results7["Kfold"].append(50)
results7["Scaler"].append(scaler)



results7["TEST_MSE"].append(f"{testM_mse:.2f} ± {testD_mse:.2f}")
results7["TRAIN_MSE"].append(f"{trainM_mse:.2f} ± {trainD_mse:.2f}")

predict = grid_search.predict(X_test)

df7 = pd.DataFrame(results7)

df7.to_excel(result_path+'RF_Regression.xlsx', index = False)

df_p = pd.DatFrame(predict)
df_p.to_excel(result_path+'RF_Prediction.xlsx')

Feature_selction: rr
TEST MSE : 379.5443111461145 ± 105.47004479021965
TRAIN MSE : 160.0384681826675 ± 51.948892095537936
1


KeyError: 'Approach'

In [ ]:
results7 = {'Approach':[],
            "Regressor":[],
    "Feature_Selection":[],
    'Kfold':[],
    'Scaler' :[],
    'TEST_MSE':[],
    'TRAIN_MSE': []
}
results7['Approach']. append(False)
results7["Regressor"].append(reg_name)
results7["Feature_Selection"].append('rr')
results7["Kfold"].append(50)
results7["Scaler"].append(scaler)



results7["TEST_MSE"].append(f"{testM_mse:.2f} ± {testD_mse:.2f}")
results7["TRAIN_MSE"].append(f"{trainM_mse:.2f} ± {trainD_mse:.2f}")

predict = grid_search.predict(X_test)

df7 = pd.DataFrame(results7)

df7.to_excel(result_path+'RF_Regression.xlsx', index = False)

df_p = pd.DataFrame(predict)
df_p.to_excel(result_path+'RF_Prediction.xlsx')